In [ ]:
import os

task = "ner"
run_name = "roberta-base-biomedical-clinical-es-bilstm-crf_v1" # letters, numbers, dashes, underscores, dots.
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
notebooks_dir_path = "notebook_dir" # dir of this notebook
custom_dataset_path = "dataset_path/custom_dataset"
output_path = 'output_path'

# save the model at the end of the training. Use with Trainer load_best_model_at_end=True to save the best model.
os.environ["WANDB_LOG_MODEL"] = "end" # set to "false" to prevent saving.

In [ ]:
# If running in Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install datasets transformers accelerate seqeval wandb

In [ ]:
!pip uninstall -y pytorch-crf
!pip install pytorch-crf

In [ ]:
!wandb login # Click the authorize link first. The login box is just right on the last row, after the :

In [ ]:
import wandb
from datetime import datetime
import pytz

timestamp = '{date:%Y-%m-%d-%H_%M_%S}'.format(date=datetime.now(pytz.timezone('Europe/Sofia'))) # set your timezone
full_run_name = f"{run_name}_{timestamp}"

wandb_run = wandb.init(project="SympTEMIST 2023 - NER", name=full_run_name, save_code=True)

In [ ]:
import transformers
print(transformers.__version__)

# Fine-tuning a model on a token classification task

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
from datasets import DatasetDict

dataset_split = load_dataset(custom_dataset_path, split=[
    'train[0%:100%]', 'validation[0%:100%]'
], cache_dir = '.')

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

In [ ]:
datasets

In [ ]:
datasets["train"][0]

In [ ]:
# Save the datasets to WANDB
dataset_split[0].save_to_disk("./train")
dataset_split[1].save_to_disk("./validation")

datasets_artifact = wandb.Artifact(name="datasets", type="dataset")
datasets_artifact.add_dir(local_path="./train", name="train")  # 'name' specifies a directory in the artifact
datasets_artifact.add_dir(local_path="./validation", name="validation")
wandb_run.log_artifact(datasets_artifact) # versioning is automatic, starting from v0

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## Fine-tuning the model

In [ ]:
import torch
from torch import nn
from torchcrf import CRF
from transformers import RobertaPreTrainedModel, RobertaModel, TrainingArguments, Trainer
from transformers.modeling_outputs import  TokenClassifierOutput

class RobertaWithBilstmCRF(RobertaPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.lstm_dim = config.hidden_size // 2 # this is the output of a single LSTM layer. BiLSTM output = 2 * lstm_dim
        self.lstm_layers = 2

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.bilstm = nn.LSTM(config.hidden_size, self.lstm_dim, dropout=config.hidden_dropout_prob, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.crf = CRF(num_tags=self.num_labels, batch_first=True)
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss. Indices should be in ``[0, ..., config.num_labels -
            1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            mask = torch.where(labels == -100, torch.tensor(0), torch.tensor(1))
            mask[:, 0] = 1 # shameless hack
            mask = mask.bool()
            hacked_labels = torch.where(labels == -100, torch.tensor(0), labels) # another one, thanks https://github.com/kmkurn/pytorch-crf/issues/41
            log_likelihood, tags = self.crf(logits, hacked_labels, mask=mask), self.crf.decode(logits)
            loss = 0 - log_likelihood
        else:
            tags = self.crf.decode(logits)
        tags = torch.Tensor(tags)

        output = (tags,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [ ]:
model = RobertaWithBilstmCRF.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"{output_path}/{full_run_name}",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    num_train_epochs=20,
    weight_decay=0.1,
    save_strategy="epoch",
    push_to_hub=False,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    run_name=full_run_name # this is just to have a decent model name in WANDB, instead of an auto-generated one
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    # print("P object: ", p)
    predictions, labels = p
    print("Predictions: ", predictions)
    print("Labels: ", labels)
    print("Predictions shape: ", predictions.shape)
    print("Labels shape: ", labels.shape)
    print("Predictions dtype: ", predictions.dtype)

    # predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[int(p)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[int(l)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    pd.DataFrame(data={"predictions": true_predictions, "labels": true_labels}).to_csv("predictions_dump.csv", index=False)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

In [ ]:
# Save the current notebook code to WANDB
# TODO: This is hacky, but WANDB cannot find the notebook file for some reason, when running in Colab.
# It generally has problems with files outside PWD, might be because of the virtual drive
# See https://github.com/wandb/wandb/issues/5169
from urllib.parse import unquote
nb_name = unquote(wandb.jupyter.notebook_metadata_from_jupyter_servers_and_kernel_id()["name"])

code_artifact = wandb.Artifact(name="notebook", type="code")
code_artifact.add_file(os.path.join(notebooks_dir_path, nb_name))
wandb_run.log_artifact(code_artifact)

# wandb_run.log_code(notebooks_dir_path, include_fn=lambda path: path.endswith(nb_name))

In [ ]:
wandb_run.finish()

### How to load a model from WANDB

In [ ]:
# # How to use an uploaded WANDB model

# import wandb
# from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

# # Create a new run
# with wandb.init(project="SympTEMIST 2023 - NER") as run:

#   # Connect an Artifact to the run
#   my_model_name = "model-spanish-roberta-baseline_2023-09-16-17_02_11:latest"
#   my_model_artifact = run.use_artifact(my_model_name)

#   # Download model weights to a folder and return the path
#   model_dir = my_model_artifact.download()

#   # Load your Hugging Face model from that folder
#   #  using the same model class
#   model = AutoModelForTokenClassification.from_pretrained(
#       model_dir, num_labels=3)

#   tokenizer = AutoTokenizer.from_pretrained(model_dir)

#   # Do additional training, or run inference